In [ ]:
from __future__ import annotations
import os, sys, json, time, subprocess
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Iterable
from collections import defaultdict

In [ ]:
def ensure_deps():
    pkgs = ["torch", "faiss-cpu", "sentence-transformers"]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
try:
    import torch, faiss
    from sentence_transformers import SentenceTransformer
except Exception:
    ensure_deps()
    import torch, faiss
    from sentence_transformers import SentenceTransformer

import numpy as np

In [ ]:
@dataclass
class EncoderConfig:
    model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    max_seq_length: int = 384
    batch_size: int = 64
    device: Optional[str] = None

@dataclass
class BuildPaths:
    out_dir: str
    chunk_faiss: str
    chunk_docstore: str
    chunk_embeddings: str
    manifest: str
    numbers_faiss: str
    numbers_meta: str

def make_paths(out_dir: str) -> BuildPaths:
    p = Path(out_dir); p.mkdir(parents=True, exist_ok=True)
    return BuildPaths(
        out_dir=str(p),
        chunk_faiss=str(p / "chunks.faiss"),
        chunk_docstore=str(p / "docstore.json"),
        chunk_embeddings=str(p / "X.npy"),
        manifest=str(p / "manifest.json"),
        numbers_faiss=str(p / "numbers.faiss"),
        numbers_meta=str(p / "numbers_meta.json"),
    )

In [ ]:
def load_chunks_jsonl(path: str) -> List[Dict]:
    items = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            items.append(json.loads(line))
    return items

In [ ]:
class STEncoder:
    def __init__(self, cfg: EncoderConfig):
        dev = cfg.device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(cfg.model_name, device=dev)
        self.model.max_seq_length = cfg.max_seq_length
        self.batch_size = cfg.batch_size

    def encode(self, texts: List[str]) -> np.ndarray:
        try:
            embs = self.model.encode(
                texts,
                batch_size=self.batch_size,
                convert_to_numpy=True,
                normalize_embeddings=True,
                show_progress_bar=True,
            ).astype(np.float32)
        except TypeError:
            embs = self.model.encode(
                texts,
                batch_size=self.batch_size,
                convert_to_numpy=True,
                show_progress_bar=True,
            ).astype(np.float32)
            faiss.normalize_L2(embs)
        return embs

In [ ]:
class ChunkFaissIndex:
    def __init__(self, dim: int):
        self.dim = dim
        self.index = faiss.IndexFlatIP(dim)
        self.ids: List[str] = []
        self.texts: List[str] = []
        self.numbers: List[str] = []
        self.sections: List[str] = []

    def add(self, X: np.ndarray, ids: List[str], texts: List[str], numbers: List[str], sections: List[str]):
        assert X.shape[0] == len(ids) == len(texts) == len(numbers) == len(sections)
        faiss.normalize_L2(X)
        self.index.add(X.astype(np.float32))
        self.ids, self.texts, self.numbers, self.sections = ids, texts, numbers, sections

    def search(self, Q: np.ndarray, top_k: int) -> Tuple[np.ndarray, np.ndarray]:
        faiss.normalize_L2(Q)
        sims, idxs = self.index.search(Q.astype(np.float32), top_k)
        return sims, idxs

    def save(self, paths: BuildPaths, manifest: Dict):
        faiss.write_index(self.index, paths.chunk_faiss)
        with open(paths.chunk_docstore, "w", encoding="utf-8") as f:
            json.dump(
                {"ids": self.ids, "texts": self.texts, "numbers": self.numbers, "sections": self.sections},
                f, ensure_ascii=False, indent=2
            )
        with open(paths.manifest, "w", encoding="utf-8") as f:
            json.dump(manifest, f, ensure_ascii=False, indent=2)

    @classmethod
    def load(cls, paths: BuildPaths, dim: int) -> "ChunkFaissIndex":
        idx = faiss.read_index(paths.chunk_faiss)
        doc = json.loads(Path(paths.chunk_docstore).read_text(encoding="utf-8"))
        store = cls(dim)
        store.index = idx
        store.ids = doc["ids"]; store.texts = doc["texts"]; store.numbers = doc["numbers"]; store.sections = doc["sections"]
        return store

In [ ]:
def build_number_centroids(X: np.ndarray, numbers: List[str], method: str = "mean") -> Tuple[List[str], np.ndarray, Dict[str, List[int]]]:
    groups: Dict[str, List[int]] = defaultdict(list)
    for i, num in enumerate(numbers):
        groups[num].append(i)
    unique_numbers = sorted(groups.keys(), key=lambda n: int(n) if str(n).isdigit() else n)
    C = []
    for num in unique_numbers:
        idx = groups[num]
        M = X[idx]
        if method == "max":
            v = M.max(axis=0)
        else:
            v = M.mean(axis=0)
        v = v / np.linalg.norm(v).clip(min=1e-9)
        C.append(v.astype(np.float32))
    C = np.vstack(C).astype(np.float32)
    faiss.normalize_L2(C)
    return unique_numbers, C, groups

In [ ]:
class NumberFaissIndex:
    def __init__(self, dim: int):
        self.dim = dim
        self.index = faiss.IndexFlatIP(dim)
        self.numbers: List[str] = []
        self.groups: Dict[str, List[int]] = {}

    def build(self, C: np.ndarray, numbers: List[str], groups: Dict[str, List[int]]):
        faiss.normalize_L2(C)
        self.index.add(C.astype(np.float32))
        self.numbers = numbers
        self.groups = groups

    def search(self, Q: np.ndarray, top_k: int) -> Tuple[np.ndarray, np.ndarray]:
        faiss.normalize_L2(Q)
        sims, idxs = self.index.search(Q.astype(np.float32), top_k)
        return sims, idxs

    def save(self, paths: BuildPaths):
        faiss.write_index(self.index, paths.numbers_faiss)
        with open(paths.numbers_meta, "w", encoding="utf-8") as f:
            json.dump({"numbers": self.numbers, "groups": self.groups}, f, ensure_ascii=False, indent=2)

    @classmethod
    def load(cls, paths: BuildPaths, dim: int) -> "NumberFaissIndex":
        idx = faiss.read_index(paths.numbers_faiss)
        meta = json.loads(Path(paths.numbers_meta).read_text(encoding="utf-8"))
        store = cls(dim)
        store.index = idx
        store.numbers = meta["numbers"]
        store.groups = {k: list(v) for k, v in meta["groups"].items()}
        return store

In [ ]:
def build_all_indices(
    chunks_jsonl: str,
    out_dir: str,
    enc_cfg: EncoderConfig = EncoderConfig(),
    include_embeddings_cache: bool = True,
    centroid_method: str = "mean"
) -> Tuple[ChunkFaissIndex, NumberFaissIndex]:
    paths = make_paths(out_dir)
    enc = STEncoder(enc_cfg)

    rows = load_chunks_jsonl(chunks_jsonl)
    ids = [r["id"] for r in rows]
    texts = [r["text"] for r in rows]
    numbers = [r["number"] for r in rows]
    sections = [r["section"] for r in rows]

    if include_embeddings_cache and Path(paths.chunk_embeddings).exists():
        X = np.load(paths.chunk_embeddings).astype(np.float32)
    else:
        X = enc.encode(texts).astype(np.float32)
        if include_embeddings_cache:
            np.save(paths.chunk_embeddings, X)

    chunk_idx = ChunkFaissIndex(dim=X.shape[1])
    chunk_idx.add(X, ids, texts, numbers, sections)
    manifest = {
        "model_name": enc_cfg.model_name,
        "dim": X.shape[1],
        "max_seq_length": enc_cfg.max_seq_length,
        "built_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "chunks_source": str(chunks_jsonl),
        "count": len(texts),
    }
    chunk_idx.save(paths, manifest)

    num_list, C, groups = build_number_centroids(X, numbers, method=centroid_method)
    number_idx = NumberFaissIndex(dim=C.shape[1])
    number_idx.build(C, num_list, groups)
    number_idx.save(paths)

    print(f"Built chunk index ({len(texts)} vecs) and number index ({len(num_list)} numbers) → {out_dir}")
    return chunk_idx, number_idx

In [ ]:
SECTION_WEIGHT = {
    "enunciado": 1.10,
    "enunciado_mini": 1.10,
    "referencias_legislativas": 1.05,
    "excertos_precedentes": 1.00,
    "orgao_data_fonte": 0.95,
    "header": 0.90,
}

In [ ]:
def aggregate_by_number(
    chunk_ids: List[str],
    chunk_numbers: List[str],
    chunk_sections: List[str],
    sims: np.ndarray,
    pool: str = "sum_sqrt",
) -> Dict[str, float]:
    agg = defaultdict(float)
    counts = defaultdict(int)
    for j, s in zip(chunk_ids, sims):
        sec = chunk_sections[j]
        num = chunk_numbers[j]
        w = SECTION_WEIGHT.get(sec, 1.0)
        agg[num] += float(s) * w
        counts[num] += 1
    if pool == "sum_sqrt":
        for n in list(agg.keys()):
            agg[n] = agg[n] / max(1.0, np.sqrt(counts[n]))
    elif pool == "max":
        tmp = defaultdict(float)
        for j, s in zip(chunk_ids, sims):
            sec = chunk_sections[j]; num = chunk_numbers[j]
            w = SECTION_WEIGHT.get(sec, 1.0)
            tmp[num] = max(tmp[num], float(s) * w)
        agg = tmp
    return agg

In [1]:
class PerNumberRetriever:
    def __init__(self, out_dir: str):
        paths = make_paths(out_dir)
        manifest = json.loads(Path(paths.manifest).read_text(encoding="utf-8"))
        dim = manifest["dim"]
        self.chunk_idx = ChunkFaissIndex.load(paths, dim=dim)
        self.number_idx = None
        if Path(paths.numbers_faiss).exists():
            self.number_idx = NumberFaissIndex.load(paths, dim=dim)

    def encode_queries(self, texts: List[str], enc_cfg: EncoderConfig) -> np.ndarray:
        enc = STEncoder(enc_cfg)
        return enc.encode(texts).astype(np.float32)

    def search_chunk_only(
        self,
        Q: np.ndarray,
        enc_cfg: EncoderConfig,
        top_chunks: int = 200,
        pool: str = "sum_sqrt",
        ignore_ids: Optional[List[str]] = None,
    ) -> List[List[Tuple[str, float]]]:
        sims, idxs = self.chunk_idx.search(Q, top_k=top_chunks)
        results = []
        ignore = set(ignore_ids or [])
        for qi in range(Q.shape[0]):
            js, ss = [], []
            for r, j in enumerate(idxs[qi]):
                if j == -1:
                    continue
                if self.chunk_idx.ids[j] in ignore:
                    continue
                js.append(j); ss.append(sims[qi, r])
            agg = aggregate_by_number(
                chunk_ids=js,
                chunk_numbers=self.chunk_idx.numbers,
                chunk_sections=self.chunk_idx.sections,
                sims=np.array(ss),
                pool=pool,
            )
            ranked = sorted(agg.items(), key=lambda x: x[1], reverse=True)
            results.append(ranked)
        return results

    def search_two_stage(
        self,
        Q: np.ndarray,
        enc_cfg: EncoderConfig,
        top_numbers: int = 20,
        top_chunks_per_query: int = 200,
        pool: str = "sum_sqrt",
        ignore_ids: Optional[List[str]] = None,
    ) -> List[List[Tuple[str, float]]]:
        if self.number_idx is None:
            return self.search_chunk_only(Q, enc_cfg, top_chunks=top_chunks_per_query, pool=pool, ignore_ids=ignore_ids)
        simsN, idxsN = self.number_idx.search(Q, top_k=top_numbers)
        allowed_sets: List[set] = []
        for qi in range(Q.shape[0]):
            nums = [self.number_idx.numbers[j] for j in idxsN[qi] if j != -1]
            allowed_sets.append(set(nums))
        simsC, idxsC = self.chunk_idx.search(Q, top_k=top_chunks_per_query)
        results = []
        ignore = set(ignore_ids or [])
        for qi in range(Q.shape[0]):
            js, ss = [], []
            for r, j in enumerate(idxsC[qi]):
                if j == -1:
                    continue
                if self.chunk_idx.ids[j] in ignore:
                    continue
                if self.chunk_idx.numbers[j] not in allowed_sets[qi]:
                    continue
                js.append(j); ss.append(simsC[qi, r])
            agg = aggregate_by_number(
                chunk_ids=js,
                chunk_numbers=self.chunk_idx.numbers,
                chunk_sections=self.chunk_idx.sections,
                sims=np.array(ss),
                pool=pool,
            )
            ranked = sorted(agg.items(), key=lambda x: x[1], reverse=True)
            results.append(ranked)
        return results
    def pick_evidence(
        self,
        number: str,
        top_chunks_global_idxs: Iterable[int],
        max_chunks: int = 3
    ) -> List[Dict]:
        prefer = ["enunciado", "enunciado_mini", "excertos_precedentes", "referencias_legislativas", "orgao_data_fonte", "header"]
        selected = []
        seen = set()
        for sec in prefer:
            for j in top_chunks_global_idxs:
                if self.chunk_idx.numbers[j] == number and self.chunk_idx.sections[j] == sec and j not in seen:
                    selected.append({
                        "id": self.chunk_idx.ids[j],
                        "section": self.chunk_idx.sections[j],
                        "text": self.chunk_idx.texts[j],
                    })
                    seen.add(j)
                    if len(selected) >= max_chunks:
                        return selected
        return selected


In [2]:


from pathlib import Path

enc_cfg = EncoderConfig(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  # or MiniLM
    max_seq_length=384,
    batch_size=64,
    device=None,
)
out_dir = "/content/faiss_per_number"
if not (Path(out_dir) / "chunks.faiss").exists():
    chunk_idx, number_idx = build_all_indices(
        chunks_jsonl="/content/sumulas_chunks_mpnet.jsonl",
        out_dir=out_dir,
        enc_cfg=enc_cfg,
        include_embeddings_cache=True,
        centroid_method="mean"
    )

retriever = PerNumberRetriever(out_dir=out_dir)

QUERY = """PRISÃO PREVENTIVA. PORTE ILEGAL DE ARMA DE FOGO DE USO PERMITIDO, TRÁFICO
DE DROGAS E ASSOCIAÇÃO PARA O TRÁFICO. GARANTIA DA ORDEM PÚBLICA. CONVERSÃO EX
OFFICIO DA PRISÃO EM FLAGRANTE EM PREVENTIVA. IMPOSSIBILIDADE. NECESSIDADE DE
REQUERIMENTO PRÉVIO OU PELO MINISTÉRIO PÚBLICO OU PELO QUERELANTE, OU PELO
ASSISTENTE OU, POR FIM, MEDIANTE REPRESENTAÇÃO DA AUTORIDADE POLICIAL. [...] No
caso, a decisão agravada deve ser mantida, uma vez que não é possível a
decretação da prisão preventiva de ofício em face do que dispõe a Lei n.
13.964/2019, mesmo se decorrente de prisão em flagrante e se não tiver ocorrido
audiência de custódia. Isso porque não existe diferença entre a conversão da
prisão em flagrante em preventiva e a decretação da prisão preventiva como uma
primeira prisão (EDcl no AgRg no HC n. 653.425/MG, de minha relatoria, Sexta
Turma, DJe 19/11/2021)"""
QUERY = " ".join(QUERY.split())

Q = retriever.encode_queries([QUERY], enc_cfg=enc_cfg)

ranked_numbers = retriever.search_two_stage(
    Q, enc_cfg,
    top_numbers=20,
    top_chunks_per_query=200,
    pool="sum_sqrt"
)[0]

print("Top súmulas (number, score):")
for n, s in ranked_numbers[:10]:
    print(f"- {n}: {s:.4f}")

sims, idxs = retriever.chunk_idx.search(Q, top_k=200)
top_number = ranked_numbers[0][0]
evidence = retriever.pick_evidence(number=top_number, top_chunks_global_idxs=idxs[0], max_chunks=3)

print("\nEvidence (id, section):")
for e in evidence:
    print(f"- {e['id']} | {e['section']}")
    preview = e["text"].replace("\n", " ")
    print("  ", preview[:280] + ("…" if len(preview) > 280 else ""))

enunciados = [
    retriever.chunk_idx.texts[i]
    for i, num in enumerate(retriever.chunk_idx.numbers)
    if num == top_number and retriever.chunk_idx.sections[i] == "enunciado"
]
if enunciados:
    print(f"\nEnunciado da Súmula {top_number}:")
    print(enunciados[0])
try:
    topN_nums = [int(n) for n, _ in ranked_numbers[:10] if str(n).isdigit()]
    if topN_nums:
        print("\nMaior número dentro do top-10:", max(topN_nums))
except Exception:
    pass


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/91 [00:00<?, ?it/s]

Built chunk index (5764 vecs) and number index (676 numbers) → /content/faiss_per_number


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top súmulas (number, score):
- 676: 1.6910
- 440: 1.6867
- 617: 1.6647
- 439: 1.6557
- 491: 1.6493
- 415: 1.6388
- 347: 1.6368
- 643: 1.6179
- 471: 1.6089
- 441: 1.4448

Evidence (id, section):
- 676#excertos_precedentes@272-592 | excertos_precedentes
   SÚMULA 676 — EXCERTOS DOS PRECEDENTES DROGAS E ASSOCIAÇÃO PARA O TRÁFICO. GARANTIA DA ORDEM PÚBLICA. CONVERSÃO EX OFFICIO DA PRISÃO EM FLAGRANTE EM PREVENTIVA. IMPOSSIBILIDADE. NECESSIDADE DE REQUERIMENTO PRÉVIO OU PELO MINISTÉRIO PÚBLICO OU PELO QUERELANTE, OU PELO ASSISTENTE OU…
- 676#excertos_precedentes@544-864 | excertos_precedentes
   SÚMULA 676 — EXCERTOS DOS PRECEDENTES , DJe de 31/3/2022) "[...] TRÁFICO DE DROGAS. REQUERIMENTO DO MINISTÉRIO PÚBLICO PARA APLICAÇÃO DE MEDIDAS CAUTELARES MAIS BRANDAS. DECRETAÇÃO DA PRISÃO PREVENTIVA. ATUAÇÃO DE OFÍCIO. CONSTRANGIMENTO ILEGAL CONFIGURADO. [...] A reforma introd…
- 676#excertos_precedentes@816-1136 | excertos_precedentes
   SÚMULA 676 — EXCERTOS DOS PRECEDENTES audiência de custódi

In [3]:
def predict_sumula(text: str, retriever, enc_cfg, top_numbers=20, top_chunks=200, max_evidence=3):
    Q = retriever.encode_queries([text], enc_cfg=enc_cfg)
    ranked = retriever.search_two_stage(Q, enc_cfg, top_numbers=top_numbers,
                                        top_chunks_per_query=top_chunks, pool="sum_sqrt")[0]
    if not ranked:
        return None
    sumula, score = ranked[0]
    sims, idxs = retriever.chunk_idx.search(Q, top_k=top_chunks)
    evidence = retriever.pick_evidence(sumula, idxs[0], max_chunks=max_evidence)

    enunciado = next(
        (retriever.chunk_idx.texts[i] for i, num in enumerate(retriever.chunk_idx.numbers)
         if num == sumula and retriever.chunk_idx.sections[i] == "enunciado"),
        None
    )
    return {
        "number": sumula,
        "score": float(score),
        "enunciado": enunciado,
        "evidence": [{"id": e["id"], "section": e["section"], "text": e["text"]} for e in evidence],
        "candidates": ranked[:5],
    }


In [6]:
from pathlib import Path
from typing import List, Dict, Any

enc_cfg = EncoderConfig(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    max_seq_length=384,
    batch_size=64,
    device=None,
)
OUT_DIR = "/content/faiss_per_number"

if not (Path(OUT_DIR) / "chunks.faiss").exists():
    _chunk_idx, _number_idx = build_all_indices(
        chunks_jsonl="/content/sumulas_chunks_mpnet.jsonl",
        out_dir=OUT_DIR,
        enc_cfg=enc_cfg,
        include_embeddings_cache=True,
        centroid_method="mean",
    )

retriever = PerNumberRetriever(out_dir=OUT_DIR)

def _enunciado_for_number(retriever: PerNumberRetriever, number: str) -> str:
    for i, num in enumerate(retriever.chunk_idx.numbers):
        if num == number and retriever.chunk_idx.sections[i] == "enunciado":
            return retriever.chunk_idx.texts[i]
    return ""

def predict_sumula(text: str,
                   top_numbers: int = 20,
                   top_chunks_per_query: int = 200,
                   max_evidence: int = 3,
                   pool: str = "sum_sqrt") -> Dict[str, Any]:
    Q = retriever.encode_queries([text], enc_cfg=enc_cfg)
    ranked = retriever.search_two_stage(
        Q, enc_cfg,
        top_numbers=top_numbers,
        top_chunks_per_query=top_chunks_per_query,
        pool=pool
    )[0]
    if not ranked:
        return {"number": None, "score": 0.0, "enunciado": "", "evidence": [], "candidates": []}

    number, score = ranked[0]
    sims, idxs = retriever.chunk_idx.search(Q, top_k=top_chunks_per_query)
    evidence = retriever.pick_evidence(number=number, top_chunks_global_idxs=idxs[0], max_chunks=max_evidence)
    return {
        "number": number,
        "score": float(score),
        "enunciado": _enunciado_for_number(retriever, number),
        "evidence": [{"id": e["id"], "section": e["section"], "text": e["text"]} for e in evidence],
        "candidates": ranked[:5],
    }

def predict_many(queries: List[str],
                 top_numbers: int = 20,
                 top_chunks_per_query: int = 200,
                 max_evidence: int = 3,
                 pool: str = "sum_sqrt") -> List[Dict[str, Any]]:
    Q = retriever.encode_queries(queries, enc_cfg=enc_cfg)
    ranked_all = retriever.search_two_stage(
        Q, enc_cfg,
        top_numbers=top_numbers,
        top_chunks_per_query=top_chunks_per_query,
        pool=pool
    )
    sims, idxs = retriever.chunk_idx.search(Q, top_k=top_chunks_per_query)
    results = []
    for qi, ranked in enumerate(ranked_all):
        if not ranked:
            results.append({"number": None, "score": 0.0, "enunciado": "", "evidence": [], "candidates": []})
            continue
        number, score = ranked[0]
        evidence = retriever.pick_evidence(number=number, top_chunks_global_idxs=idxs[qi], max_chunks=max_evidence)
        results.append({
            "number": number,
            "score": float(score),
            "enunciado": _enunciado_for_number(retriever, number),
            "evidence": [{"id": e["id"], "section": e["section"], "text": e["text"]} for e in evidence],
            "candidates": ranked[:5],
        })
    return results

def pretty_print(result: Dict[str, Any], max_preview_chars: int = 260) -> None:
    print(f"\nPredicted Súmula: {result['number']}  |  score={result['score']:.4f}")
    if result["enunciado"]:
        enun = result["enunciado"].replace("\n", " ")
        print("Enunciado:", (enun[:max_preview_chars] + "…") if len(enun) > max_preview_chars else enun)
    if result["evidence"]:
        print("Evidence:")
        for e in result["evidence"]:
            prev = e["text"].replace("\n", " ")
            print(f" - {e['id']} [{e['section']}]:", (prev[:max_preview_chars] + "…") if len(prev) > max_preview_chars else prev)
    if result["candidates"]:
        print("Top-5:", result["candidates"])


In [7]:
res = predict_sumula(QUERY)
pretty_print(res)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Predicted Súmula: 676  |  score=1.6910
Enunciado: SÚMULA 676 — ENUNCIADO Enunciado: Em razão da Lei n. 13.964/2019, não é mais possível ao juiz, de ofício, decretar ou converter prisão em flagrante em prisão preventiva.
Evidence:
 - 676#excertos_precedentes@272-592 [excertos_precedentes]: SÚMULA 676 — EXCERTOS DOS PRECEDENTES DROGAS E ASSOCIAÇÃO PARA O TRÁFICO. GARANTIA DA ORDEM PÚBLICA. CONVERSÃO EX OFFICIO DA PRISÃO EM FLAGRANTE EM PREVENTIVA. IMPOSSIBILIDADE. NECESSIDADE DE REQUERIMENTO PRÉVIO OU PELO MINISTÉRIO PÚBLICO OU PELO QUERELANTE, O…
 - 676#excertos_precedentes@544-864 [excertos_precedentes]: SÚMULA 676 — EXCERTOS DOS PRECEDENTES , DJe de 31/3/2022) "[...] TRÁFICO DE DROGAS. REQUERIMENTO DO MINISTÉRIO PÚBLICO PARA APLICAÇÃO DE MEDIDAS CAUTELARES MAIS BRANDAS. DECRETAÇÃO DA PRISÃO PREVENTIVA. ATUAÇÃO DE OFÍCIO. CONSTRANGIMENTO ILEGAL CONFIGURADO. [.…
 - 676#excertos_precedentes@816-1136 [excertos_precedentes]: SÚMULA 676 — EXCERTOS DOS PRECEDENTES audiência de custódia, a c

In [ ]:
QUERY = """"[...] CRÉDITO RURAL. SECURITIZAÇÃO. ALONGAMENTO DA DÍVIDA RURAL. LEI 9.138/95. DIREITO DO
MUTUÁRIO. [...] É direito do devedor, desde que atendidos os requisitos estipulados na lei 9.138/95, o
alongamento das dívidas originárias de crédito rural. II. Reconhecido o direito acima, compete às
instâncias ordinárias a verificação do atendimento dos requisitos autorizadores da securitização
postulada. [...]" (REsp 234246 SP, Rel. Ministro ALDIR PASSARINHO JUNIOR, QUARTA TURMA, julgado
em 29/08/2000, DJ 13/11/2000, p. 146)"""
QUERY = " ".join(QUERY.split())

In [11]:
res = predict_sumula(QUERY)
pretty_print(res)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Predicted Súmula: 298  |  score=1.9628
Enunciado: SÚMULA 298 — ENUNCIADO Enunciado: O alongamento de dívida originada de crédito rural não constitui faculdade da instituição financeira, mas, direito do devedor nos termos da lei.
Evidence:
 - 298#enunciado [enunciado]: SÚMULA 298 — ENUNCIADO Enunciado: O alongamento de dívida originada de crédito rural não constitui faculdade da instituição financeira, mas, direito do devedor nos termos da lei.
 - 298#enunciado_mini@0-29 [enunciado_mini]: SÚMULA 298 — ENUNCIADO (MINI) O alongamento de dívida originada de crédito rural não constitui faculdade da instituição financeira, mas, direito do devedor nos termos da lei.
 - 298#excertos_precedentes@544-864 [excertos_precedentes]: SÚMULA 298 — EXCERTOS DOS PRECEDENTES vez preenchidos os seus requisitos, o alongamento das dívidas rurais, e não permitiu simples faculdade a ser usada discricionariamente pela instituição de crédito'. 2. Afastado o óbice do direito à securitização, as instân…
Top-5: [(